# Cloud Services Cookbook

This notebook will review how to connect to and make use of cloud service providers.

* Amazon S3
* Dropbox
* Twitter

# Amazon Web Services (AWS)

AWS is Amazon's cloud framework. [boto](https://aws.amazon.com/sdk-for-python/) is the AWS SDK for Python.  boto provides Python APIs for many AWS services including Amazon S3, Amazon EC2, Amazon DynamoDB, and more. It supports Python 2 and 3.

In [ ]:
#<help:amazon_help>

### Setup
Install the [boto](https://aws.amazon.com/sdk-for-python/) Python SDK library, which wraps the Amazon REST APIs.

In [ ]:
#<help:amazon_setup>
!pip install boto

In [ ]:
#<help:amazon_setup>
!pip install filechunkio

## Amazon S3
Amazon Simple Storage Service (S3) provides file storage in the cloud.  Files are encapsulated as objects.  Amazon S3 enables developers to store, retrieve and delete objects. It also allows setting permissions and metadata on files through the object interfaces.

To be able to perform operations on Amazon S3, you must first have an AWS account with S3 enabled.  You must also provide your AWS credentials in order to connect.  You can obtain your credentials by following these steps: 

1. Login to [AWS](https://portal.aws.amazon.com/billing/signup).
2. Click `Account`.
3. Click the `Security Credentials` link.
4. Extract the key ID and secret access key. (You will use them in the code cells below to access and connect to Amazon S3).
5. Grant Access to read/write on buckets and its contents.  
    * Go to [AWS->IAM Management Console](https://console.aws.amazon.com/iam/)
    * Click `Dashboard` > `Users` and select the user. 
    * Click on `User Policies` and add `PowerUserAccess` and `AdministratorAccess`.

In [ ]:
#<help:amazon_s3_help>

###Create Bucket
Create a [bucket](http://docs.aws.amazon.com/AmazonS3/latest/dev/UsingBucket.html), the container used in Amazon S3 for data storage, and put an object in that bucket.

In [ ]:
#<help:amazon_s3_create_bucket>
import boto
import time

aws_access_key_id = "ACCESS KEY ID"
aws_secret_access_key = "SECRET ACCESS KEY"
s3 = boto.connect_s3(aws_access_key_id, aws_secret_access_key)

# Create a new bucket. Buckets must have a globally unique name (not just
# unique to the account, so choose a good name).
bucket = s3.create_bucket("globally_unique_name_for_s3_bucket")
# Create a new key/value pair.
key = bucket.new_key('mykey')
key.set_contents_from_string("Hello World!")
# Sleep to ensure the data is eventually there.
time.sleep(2)
# Retrieve the contents of the key.
print key.get_contents_as_string()
# Delete key.
key.delete()
# Delete bucket.
bucket.delete()

### Store Data with Keys  
Any type of data file can be stored in Amazon S3.  Each file is stored as a key/value pair, where the key is unique within a bucket.  

The `Key` object is used in `boto` to keep track of data stored in Amazon S3.  Store new data in Amazon S3 by creating a new `Key` object and adding a value:

In [ ]:
#<help:amazon_s3_get_key>
import boto
from boto.s3.key import Key
import time 

aws_access_key_id = "ACCESS KEY ID"
aws_secret_access_key = "SECRET ACCESS KEY"
s3 = boto.connect_s3(aws_access_key_id, aws_secret_access_key)
b = s3.create_bucket('globally_unique_name_for_s3_bucket')
k = b.new_key('newS3key')
k.set_contents_from_string('This is a test of S3.')
# Sleep to ensure the data is eventually there.
time.sleep(2)
# Retrieve contents of key.
print k.get_contents_as_string()

### Upload/Download file to/from Amazon S3
With a key, you can upload and download files to and from Amazon S3.

In [ ]:
#<help:amazon_s3_key_contents_from_filename>
import boto
from boto.s3.key import Key

aws_access_key_id = "ACCESS KEY ID"
aws_secret_access_key = "SECRET ACCESS KEY"
s3 = boto.connect_s3(aws_access_key_id, aws_secret_access_key)
b = s3.create_bucket('globally_unique_name_for_s3_bucket')
k = Key(b)
k.key = 'my test file'
# create test file on disk
testfile_name = "uploaded_aws_test_file.txt"
with open("/resources/data/" + testfile_name, 'w') as f:
    f.write("Test file content")
# upload test file to S3
k.set_contents_from_filename(testfile_name)
print('uploaded sucessfully')
# download test file from S3 with new file name
k.get_contents_to_filename('downloaded_aws_test_file.txt')
print('downloaded sucessfully')

Keys are validated to see if they exist. This can be disabled with `validate=False`

In [ ]:
#<help:amazon_s3_validate_key_false>
import boto

aws_access_key_id = "ACCESS KEY ID"
aws_secret_access_key = "SECRET ACCESS KEY"
s3 = boto.connect_s3(aws_access_key_id, aws_secret_access_key)
b = s3.get_bucket('globally_unique_name_for_s3_bucket') # substitute your bucket name here
# Will hit the API to check if it exists.
possible_key = b.get_key('newS3key') # substitute your key name here
print(possible_key)
print('\n')
# Won't hit the API.
key_we_know_is_there = b.get_key('newS3key', validate=False)
print(key_we_know_is_there)

### Upload Large Data  
Amazon S3 supports splitting large files into smaller chunks.  All chunks are uploaded and then Amazon S3 combines them into a single file.

In [ ]:
#<help:amazon_s3_upload_filechunkio>
import math, os
import boto
from filechunkio import FileChunkIO

aws_access_key_id = "ACCESS KEY ID"
aws_secret_access_key = "SECRET ACCESS KEY"
s3 = boto.connect_s3(aws_access_key_id, aws_secret_access_key)
b = s3.get_bucket('globally_unique_name_for_s3_bucket')
# Get file info
testfile_name = "uploaded_aws_test_file.txt"
with open("/resources/data/" + testfile_name, 'w') as f:
    f.write("Test file content")
# source_path = 'uploaded_aws_test_file.txt'
file_size = os.stat(testfile_name).st_size
# Create a multipart upload request
mp = b.initiate_multipart_upload(os.path.basename(testfile_name))
# Use a chunk size of 50 MiB (feel free to change this)
chunk_size = 52428800
chunk_count = int(math.ceil(file_size / chunk_size))
# Send the file parts, using FileChunkIO to create a file-like object
# that points to a certain byte range within the original file. 
# S3 set bytes to never exceed the original file size.
for i in range(chunk_count + 1):
    offset = chunk_size * i
    bytes = min(chunk_size, file_size - offset)
    with FileChunkIO(testfile_name, 'r', offset=offset,
                     bytes=bytes) as fp:
        mp.upload_part_from_file(fp, part_num=i + 1)
# Finish the upload
mp.complete_upload()
print('Upload Complete')

### Get Bucket
Get a bucket.

In [ ]:
#<help:amazon_s3_get_bucket>
import boto

aws_access_key_id = "ACCESS KEY ID"
aws_secret_access_key = "SECRET ACCESS KEY"
s3 = boto.connect_s3(aws_access_key_id, aws_secret_access_key)
mybucket = s3.get_bucket('globally_unique_name_for_s3_bucket') # Substitute in your bucket name, might need an entire path 
mybucket.list()

###Delete Bucket  
Delete a bucket.  

**Note**: you must first delete the contents of the bucket, or an error is raised.

In [ ]:
#<help:amazon_s3_delete_bucket>
import boto

aws_access_key_id = "ACCESS KEY ID"
aws_secret_access_key = "SECRET ACCESS KEY"
s3 = boto.connect_s3(aws_access_key_id, aws_secret_access_key)
full_bucket = s3.get_bucket('globally_unique_name_for_s3_bucket')
# It's full of keys. Delete them all.
for key in full_bucket.list():
    key.delete()
# The bucket is empty now. Delete it.
s3.delete_bucket('globally_unique_name_for_s3_bucket')
print('Deleted')

### List All Buckets  
Retrieve all created buckets. This returns a `ResultSet` object. The `ResultSet` can be used as a `sequence` or `list` type object to retrieve `Bucket` objects.

In [ ]:
#<help:amazon_s3_list_bucket>
import boto

aws_access_key_id = "ACCESS KEY ID"
aws_secret_access_key = "SECRET ACCESS KEY"
s3 = boto.connect_s3(aws_access_key_id, aws_secret_access_key)
#get
rs = s3.get_all_buckets()
#list them out
len(rs)
for b in rs:
    print b.name
#<listing of available buckets>
#show an individual one
b = rs[0]

###Access Control   
The Access Control List (ACL) allows setting read/write permissions on objects. There are two ways to set the `ACL` for an object:

1. Create a custom ACL that grants specific rights to specific users. At the moment, the users that are specified within grants have to be registered users of Amazon Web Services so this isn’t as useful or as general as it could be.
2. Use a “canned” access control policy. There are four canned policies defined:
    * `private`: Owner gets `FULL_CONTROL`. No one else has any access rights.
    * `public-read`: Owners gets `FULL_CONTROL` and the anonymous principal is granted `READ` access.
    * `public-read-write`: Owner gets `FULL_CONTROL` and the anonymous principal is granted `READ` and `WRITE` access.
    * `authenticated-read`: Owner gets `FULL_CONTROL` and any principal authenticated as a registered Amazon S3 user is granted `READ` access.

Set the bucket to `public-read` and the ACL for a key object the bucket as an argument.

In [ ]:
#<help:amazon_s3_get_acl>
import boto

aws_access_key_id = "ACCESS KEY ID"
aws_secret_access_key = "SECRET ACCESS KEY"
s3 = boto.connect_s3(aws_access_key_id, aws_secret_access_key)
# the used bucket must be previously created
b = s3.get_bucket('globally_unique_name_for_s3_bucket')
# the used key must be previously created
k = b.get_key('newS3key')
# set acl 
b.set_acl('public-read', 'newS3key')
# set acl bucket with key
k.set_acl('public-read')
# get acl 
acp = b.get_acl()
# display perms
for grant in acp.acl.grants:
    print grant.permission, grant.display_name, grant.email_address, grant.id

#Dropbox

Dropbox offers two main APIs:

The [Core API](https://www.dropbox.com/developers/core) allows read and write access on Dropbox, such as downloading /uploading files, and modifying/deleting items. 

The [DataStore API](https://www.dropbox.com/developers/datastore) is focused more on applications that connect to Dropbox to sync data such as calendars, structured data like lists etc.

In [ ]:
#<help:dropbox_help>

##Setup

Install the [dropbox](https://pypi.python.org/pypi/dropbox) Python client library, which wraps the Dropbox REST APIs.

In [ ]:
#<help:dropbox_setup>
!pip install dropbox

## Authentication

Dropbox uses Oauth to authenticate users.  To access Dropbox from a notebook, you must first register the notebook as an application to be able to authenticate.

1. Go to the [App Console](https://www.dropbox.com/developers/apps). 
2. Click on "Create App"
3. Choose "Dropbox API App", then choose the following options: 
    * What type of data does your app need to store on Dropbox?: **Files and Datastores**
    * Can your app be limited to its own folder?: **No**
    * What type of files does your app need access to?: **All file types**
4. Select a name for your app (must be unique). 
5. Click "Create App".
6. Note the app key and app secret. You will need these to authorize access.  (Don't share these with anyone). 

The Dropbox client has a class called [DropboxOAuth2FlowNoRedirect](https://www.dropbox.com/developers/core/docs/python#DropboxOAuth2FlowNoRedirect) that our notebook app can use instead of providing an HTTP callback url during the OAuth handshake.  This class presents the end user with an authorization url, from which the user can obtain an authorization code.  From the [Dropbox Documentation](https://www.dropbox.com/developers/core/start/python):
>"This URL can ask the user to authorize your app. The URL will be printed and will ask the user to press the `Enter` key to confirm that they've authorized the app. However, in real-world apps, it's recommended to automatically send the user to the authorization URL and pass in a callback URL so that the user is seamlessly redirected back to the app after pressing a button."

After providing the authorization code to the Dropbox client, the notebook can use the client to access the user's account and perform actions.

In [ ]:
#<help: dropbox_auth>
import dropbox

# Get your app key and secret from the Dropbox developer website
app_key = 'app_key'
app_secret = 'app_secret'
# oauth dance
flow = dropbox.client.DropboxOAuth2FlowNoRedirect(app_key, app_secret)
authorize_url = flow.start()
print '1. Go to: ' + authorize_url
print '2. Click "Allow" (you may have to log in first)'
print '3. Copy the authorization code.'
# click on the link, opens new window, get the code, 
# paste the code in the cell as plaintext, has a timer so be quick!
code = raw_input("Enter the authorization code here: ").strip()
# This will fail if the user enters an invalid authorization code
access_token, user_id = flow.finish(code)
#test access to core api
client = dropbox.client.DropboxClient(access_token)
print 'linked account: ', client.account_info()

##Dropbox Core API

The Core API allows not just uploading and downloading files but other options such as search, revisions, and restoring files. 
The APi has a functionality to get a printed dictionary with the results of its commands. If the command fails, the API will return an error with the details.

In [ ]:
#<help: dropbox_core_help>

###Upload File  

`put_file()` takes a path pointing to where the file should be uploaded to Dropbox, and then a file-like object or string to be uploaded there.  

**Create on your local machine a draft.txt.**  
This command will upload it as magnum-opus.txt (change to what name you wish).

<div class="alert alert-block alert-info" style="margin-top: 20px">**Note:** You must run the `dropbox_auth` step first to authorize with Dropbox. </div> 

In [ ]:
#<help: dropbox_core_put_file>
import dropbox

testfile_name = "draft.txt"
with open("/resources/data/" + testfile_name, 'w') as f:
    f.write("Test file content")
f = open('draft.txt', 'rb')
response = client.put_file('/magnum-opus.txt', f)
#print response with details
print "Uploaded:", response

###Create Folder  

`file_create_folder(path)`  
Select the path of the folder to be created. 
<div class="alert alert-block alert-info" style="margin-top: 20px">**Note:** You must run the `dropbox_auth` step first to authorize with Dropbox. </div> 

In [ ]:
#<help: dropbox_core_create_folder>
import dropbox

response = client.file_create_folder('/mypythonapp')
print "uploaded:", response

###Copy or Move File  

Copy a file with `file_copy('from_path', 'to_path')`

To move, use `file_move('from_path', 'to_path')`

<div class="alert alert-block alert-info" style="margin-top: 20px">**Note:** You must run the `dropbox_auth` step first to authorize with Dropbox. </div> 

In [ ]:
#<help: dropbox_core_file_copy>
import dropbox

response = client.file_copy('/magnum-opus.txt','/Dropbox/mypythonapp/')
print "copied:", response

###Folder metadata  

Get the info of an entire folder by using the `metadata()` call

<div class="alert alert-block alert-info" style="margin-top: 20px">**Note:** You must run `dropbox_auth` step first to authorize with Dropbox. </div> 

In [ ]:
#<help: dropbox_core_metadata>
import dropbox

folder_metadata = client.metadata('/')
print "metadata:", folder_metadata

###Download File  

The `get_file_and_metadata` method downloads the file and its metadata.  
The method also returns the metadata at its current revision. Every time a change is made to the file, the `rev` field of the file's metadata changes as well.  
By saving the revision when the file is downloaded, it will be easy to tell if it's been modified by another device and whether to choose to download the newer revision.

<div class="alert alert-block alert-info" style="margin-top: 20px">**Note:** You must run the `dropbox_auth` step first to authorize with Dropbox. </div> 

In [ ]:
#<help: dropbox_core_get_file_and_metadata>
import dropbox

f, metadata = client.get_file_and_metadata('/magnum-opus.txt')
out = open('magnum-opus.txt', 'wb')
out.write(f.read())
out.close()
print metadata

###Search files  

`search(path, query, file_limit=1000, include_deleted=False)`  

In which  
* `path`: folder to search  
* `query`: minimum 3 characters, what to look for, case sensitive.  
* `file_limit`: up to 1000 files to display  
* `include_deleted`: whether to include deleted files in the search results  

<div class="alert alert-block alert-info" style="margin-top: 20px">**Note:** You must run the `dropbox_auth` step first to authorize with Dropbox. </div> 

In [ ]:
#<help: dropbox_core_search>
import dropbox

response = client.search('/','magnum', file_limit=100, include_deleted= False)
print "Results:", response

###Share File/Folder  

`share(path, short_url=True)`

Create a shareable link to a file or folder.

<div class="alert alert-block alert-info" style="margin-top: 20px">**Note:** You must run the `dropbox_auth` step first to authorize with Dropbox. </div> 

In [ ]:
#<help: dropbox_core_share>
import dropbox

response = client.share('/magnum-opus.txt', short_url=True)
print "Results:", response

###Upload Large Files

Upload large files with the `ChunkedUploader` object. It will break up the file into chunks and upload it in bits. 

`get_chunked_uploader(file_obj, length)`  
* `file_obj` is the file to be uploaded  
* `length`, number of bytes to upload  

Use a `try/catch` block to upload; the SDK leaves the error handling and retry logic to the developer to implement.

<div class="alert alert-block alert-info" style="margin-top: 20px">**Note:** You must run the `dropbox_auth` step first to authorize with Dropbox. </div> 

**Create a text file called big.txt**

In [ ]:
#<help:dropbox_core_chunked_uploader>
import dropbox

bigFile = open("big.txt", 'rb')
size= 1024

uploader = client.get_chunked_uploader(bigFile, size)
print "uploading: ", size
while uploader.offset < size:
    try:
        upload = uploader.upload_chunked()
        print('finished1')
    except rest.ErrorResponse, e:
            # implement error handling and retry logic here
        print('Error trying to upload')
uploader.finish('/bigFile.txt')
print('Uploaded')

###Delete file  

Delete a file or folder with `file_delete('path')`  

<div class="alert alert-block alert-info" style="margin-top: 20px">**Note:** You must run the `dropbox_auth` step first to authorize with Dropbox. </div> 

In [ ]:
#<help: dropbox_core_file_delete>
import dropbox
#delete
response = client.file_delete('/magnum-opus.txt')
print "Results:", response

##Datastore API
The Datastore API supports multiple platforms, offline access, and automatic conflict resolution.

**Client and datastore manager**  

The client lets the app start the authentication process to link with a user's Dropbox account. Once it's linked to an account (`dropbox_setup` code), the client can create a datastore manager, which can open datastores, list datastores, etc.

**Datastores and tables**  

Datastores are containers for an app's data. Each datastore contains a set of tables, and each table is a collection of records. A table allows to query existing records or insert new ones.

Any datastore with a shareable ID can be shared by assigning roles to principals, creating an access control list. Any Dropbox account with the correct permissions will then be able to open the shared datastore by ID.

**Records**  

Records are how an app stores data. Each record consists of a set of fields, each with a name and a value. Values can be simple objects, like strings, integers, and booleans, or they can be lists of simple objects. A record has an ID and can have any number of fields.

In [ ]:
#<help:dropbox_datastore_help>

###Create datastore and table 

With an access token in hand, the next step is to open the default datastore. Each app has its own default datastore per user. 
<div class="alert alert-block alert-info" style="margin-top: 20px">**Note:** You must run the `dropbox_auth` step first to authorize with Dropbox. </div> 

In [ ]:
#<help: dropbox_datastore_get_table>
import dropbox
from dropbox.datastore import DatastoreError, DatastoreManager, Date, Bytes

#define a datastore
manager = DatastoreManager(client)
datastore = manager.open_default_datastore()
print(datastore)
print('\n')
#create a table named 'tasks'
tasks_table = datastore.get_table('tasks')
print(tasks_table)

###Upload Record  

A record is a set of name and value pairs called fields. Records in the same table can have different combinations of fields; there's no schema on the table which contains them.

The Python SDK provides a method called `transaction` that uploads data to the server keeping track of conflicts.

<div class="alert alert-block alert-info" style="margin-top: 20px">**Note:** You must run the `dropbox_auth` step first to authorize with Dropbox. </div> 

In [ ]:
#<help: dropbox_datastore_record_transaction>
import dropbox
from dropbox.datastore import DatastoreError, DatastoreManager, Date, Bytes

#define a datastore
manager = DatastoreManager(client)
datastore = manager.open_default_datastore()
print(datastore)
print('\n')
#create a table named 'tasks'
tasks_table = datastore.get_table('My tasks')
print(tasks_table)
#load record and commit to dropbox with transaction
def do_insert():
    tasks_table.insert(taskname='Buy milk', completed=False)
datastore.transaction(do_insert, max_tries=4)

###Access, Edit and Delete a record  

Review uploaded datastores in the [Developer Website](https://www.dropbox.com/developers/apps/datastores)  

`.get()`, `.set()`, `.delete()`   
Access, Edit and Delete a DataStore

<div class="alert alert-block alert-info" style="margin-top: 20px">**Note:** You must run the `dropbox_auth` step first to authorize with Dropbox. </div> 

In [ ]:
#<help: dropbox_datastore_open_get_delete>
import dropbox
from dropbox.datastore import DatastoreError, DatastoreManager, Date, Bytes

#define a datastore
manager = DatastoreManager(client)
datastore = manager.open_default_datastore()
print(datastore)
print('\n')
#access a datastore
task_name = first_task.get('taskname')
#edit a datastore
def do_update():
    first_task.set('completed', True)
datastore.transaction(do_update, max_tries=4)
#delete a datastore
def do_delete():
    first_task.delete()
datastore.transaction(do_delete, max_tries=4)

###Query Record  

The query method takes a set of conditions that the fields of a record must match to be returned in the result set. All records must have a field with that name and that field's value must be exactly equal to the specified value, case sensitive too.

<div class="alert alert-block alert-info" style="margin-top: 20px">**Note:** You must run the `dropbox_auth` step first to authorize with Dropbox. </div> 

In [ ]:
#<help: dropbox_datastore_table_query>
import dropbox
from dropbox.datastore import DatastoreError, DatastoreManager, Date, Bytes

#define a datastore
manager = DatastoreManager(client)
datastore = manager.open_default_datastore()
print(datastore)
print('\n')
tasks = tasks_table.query(completed=False)
for task in tasks:
    print task.get('taskname')

###Share datastore  

The Datastore API allows sharing a datastore across multiple Dropbox accounts.

Any user who has the datastore ID and the appropriate permissions may then open the datastore.  
View the access control list at any time for a datastore as a mapping of roles applied to principals using the `list_roles()` method. Find out the current user's role with the `get_effective_role()` method.

<div class="alert alert-block alert-info" style="margin-top: 20px">**Note:** You must run the `dropbox_auth` step first to authorize with Dropbox. </div> 

In [ ]:
#<help: dropbox_datastore_share_set_role>
import dropbox
from dropbox.datastore import DatastoreError, DatastoreManager, Date, Bytes

# Shareable datastore
datastore = manager.create_datastore()
#set a role, more info in the API docs
datastore.set_role(Datastore.PUBLIC, Datastore.EDITOR)
print(datastore)
print('\n \n')
#paste data store id below!
datastore_id = 
datastore1 = manager.open_datastore(datastore_id)
print(datastore1)
print('\n \n')

**Review the Datastore Python API docs [here](https://www.dropbox.com/developers/datastore/docs/python)**.

# Twitter
Twitter offers [REST APIs](https://dev.twitter.com/rest/public) and [Streaming APIs](https://dev.twitter.com/streaming/overview) to connect and download tweets.  The documentation can be found [here](https://dev.twitter.com/overview/documentation).

The Python community has created wrappers around the Twitter API:  

* [python-twitter](https://github.com/bear/python-twitter) (Apache 2.0 License)  
* [tweepy](https://github.com/tweepy/tweepy)  (MIT License)  

We will use **tweepy** in this notebook, which wraps the [Twitter REST API](https://dev.twitter.com/rest/public).  

In [ ]:
#<help:twitter_help>

### Setup
Install the [tweepy](https://github.com/tweepy/tweepy) Python client library, which wraps the Twitter REST APIs.

In [ ]:
#<help: twitter_setup>
!pip install tweepy

### Authorize
The Twitter API uses OAuth for authentication. To allow an application to access Twitter, you must register the app on Twitter's web site.  You also need to obtain app keys and app secrets to authenticate.   

1. Go to https://apps.twitter.com/ (Sign in or create an account) 
1. Create New App. Provide the necessary information.
1. Select the application and click on "API Keys and access tokens" tab
1. Copy and paste the following into the code cell below:
    * access token/secret: Used to make API requests on your account's behalf.
    * consumer key, consumer secret: Allows the app to read and write user data.
    
First, we need to authorize to Twitter.  We only need to do this once per session.

In [ ]:
#<help: twitter_auth>
import tweepy
from tweepy import OAuthHandler

#replace with your data!
consumer_key = 'key'
consumer_secret = 'secret'
access_token = 'token'
access_token_secret = 'token_secret'
#oauth dance
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Authorize via Twitter authorization url
# And since we aren't using a web callback url, 
# We have to enter the auth code manually
redirect_url = auth.get_authorization_url()
print(redirect_url)
verifier = raw_input("Enter the authorization code here: ").strip()
# Get access token and save it
try:
    auth.get_access_token(verifier)
except tweepy.TweepError:
    print 'Error! Failed to get access token.'
auth.set_access_token(access_token, access_token_secret)

###Get Tweets
Use Twitter API retrieve tweets from our own (home) timeline.
<div class="alert alert-block alert-info" style="margin-top: 20px">**Note:** You must run `twitter_auth` step first to authorize with Twitter. </div> 

In [ ]:
#<help: twitter_tweets>

# Construct the API instance
api = tweepy.API(auth)
# Get tweets
public_tweets = api.home_timeline()
for tweet in public_tweets:
    print tweet.text

###Search Tweets  

Search the text of tweets.

<div class="alert alert-block alert-info" style="margin-top: 20px">**Note:** You must run `twitter_auth` step first to authorize with Twitter. </div> 

In [ ]:
#<help: twitter_search>
import tweepy

# Construct the API instance
api = tweepy.API(auth)
#search and iterate
public_tweets = api.search('the interview')
for tweet in public_tweets:
    print tweet.text

###List Followers 

Paginate through a user's followers.

<div class="alert alert-block alert-info" style="margin-top: 20px">**Note:** You must run `twitter_auth` step first to authorize with Twitter. </div> 

In [ ]:
#<help: twitter_lookup_users>
import tweepy
import itertools

# Construct the API instance
api = tweepy.API(auth)
#replace 'user' string with a valid Twitter handle
followers = api.followers_ids('user')

def paginate(iterable, page_size):
    while True:
        i1, i2 = itertools.tee(iterable)
        iterable, page = (itertools.islice(i1, page_size, None),
                list(itertools.islice(i2, page_size)))
        if len(page) == 0:
            break
        yield page
for page in paginate(followers, 100):
    results = api.lookup_users(user_ids=page)
    for result in results:
        print result.screen_name

###Trending Topics

Returns the top **ten** topics that are currently trending on Twitter. The response includes the time of the request, the name of each trend, and the url to the Twitter Search results page for that topic. This recipe will strip and only print the hashtags with the # symbol.

<div class="alert alert-block alert-info" style="margin-top: 20px">**Note:** You must run `twitter_auth` step first to authorize with Twitter. </div> 

In [ ]:
#<help: twitter_trends_place>
import tweepy

# Construct the API instance
api = tweepy.API(auth)
# trending topics are limited to geographical location
# use the geo code from https://developer.yahoo.com/geo/geoplanet/
trending = api.trends_place(1)
# trending is a list with only one element in it, which is a 
# dict which we'll put in data.
data = trending[0] 
# grab the trends
trends = data['trends']
# grab the name from each trend; place in a list
names = [trend['name'] for trend in trends]
# put all the names together with a ', ' separating them
trendsName = ', '.join(names)
print(trendsName)

###Cursor  

Tweepy has the `Cursor` object to iterate through timelines, user lists, direct messages, etc.
The `cursor` object returns a deserialized `json` object. 

<div class="alert alert-block alert-info" style="margin-top: 20px">**Note:** You must run `twitter_auth` step first to authorize with Twitter. </div> 

In [ ]:
#<help: twitter_cursor>
import tweepy

api = tweepy.API(auth)
tweepy.Cursor(api.user_timeline, id="twitter")
for page in tweepy.Cursor(api.user_timeline).pages():
    # page is a list of tweets, with replies
    print(page)

###List All
Get all lists that a user is subscribed to.

<div class="alert alert-block alert-info" style="margin-top: 20px">**Note:** You must run `twitter_auth` step first to authorize with Twitter. </div> 

In [ ]:
#<help: twitter_list_all>
import tweepy

api = tweepy.API(auth)
for status in api.lists_all(id = 'user_auth'):
    print pprint(status)

###Stream API

The streaming APIs allow to pull real-time data from Twitter. 

Twitter offers several streaming parameters, each customized to certain use cases.

* _Public streams_-	Streams of the public data in Twitter. Example: hashtags
* _User streams_-	Single-user streams. Example: tweets from a specific user.
* _Site streams_-	The multi-user version of user streams. 

The streaming process gets the Tweets and performs any organization needed before storing the result into a data store. 

<div class="alert alert-block alert-info" style="margin-top: 20px">**Note:** You must run `twitter_auth` step first to authorize with Twitter. </div> 

<div class="alert alert-block alert-warning" style="margin-top: 20px">**Warning** This will run indefinitely until you stop the cell. </div>

In [ ]:
#<help: twitter_stream>
import tweepy
from tweepy.streaming import StreamListener
from tweepy import Stream
import json


# This is the listener, resposible for receiving data
class StdOutListener(tweepy.StreamListener):
    def on_data(self, data):
        # Twitter returns data in JSON format - we need to decode it first
        decoded = json.loads(data)
        # Also, we convert UTF-8 to ASCII ignoring all bad characters sent by users
        print '@%s: %s' % (decoded['user']['screen_name'], decoded['text'].encode('ascii', 'ignore'))
        print ''
        return True
    def on_error(self, status):
        print status
        
print "Showing all new tweets for #python:"
stream = tweepy.Stream(auth, StdOutListener())
stream.filter(track=['python'])